# Part 4 - Altair Workflows

- **Objective**: Learn how to share and publish your Vega-Altair visualizations.
- **Key Topics**:
  - Exporting charts as HTML
  - Saving charts as static images (PNG, SVG, PDF)
  - References for embedding charts in static documents (Quarto, Jupyter Book, LaTeX)
  - References for creating dashboards

Once you have finished work on your Vega-Altair chart, how can you share it with others?

You could always use a screenshot:
* `Command+Shift+4`
* `Windows+Shift+S`

But, there are many better options, and which option you choose will depend on your specific requirements.
* Will the chart be shared as a static image in a paper or presentation?
* Are there interactive components to your chart?
* Is opening the chart in a web browser an option?

References:
* [Vega-Altair documentation](https://altair-viz.github.io/user_guide/saving_charts.html) on saving charts.
* [Discussion on GitHub](https://github.com/vega/altair/discussions/3293) from December 2023 (some new options have been added since then).


## Imports
Perform imports and create example chart for exporting.

In [ ]:
import altair as alt
import pandas as pd
import os

# Makes sure an output directory exists alongside this notebook
os.makedirs("./output", exist_ok=True)

source = pd.DataFrame({
    'a': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I'],
    'b': [28, 55, 43, 91, 81, 53, 19, 87, 52]
})

chart = alt.Chart(source).mark_bar().encode(
    x='a',
    y='b'
)
chart

## Jupyter Widget Integration
A powerful feature of Vega-Altair's support for interactivity is that it is implemented entirely in JavaScript, so these interactions work without a Python kernel (e.g. in the Vega Editor).  Sometimes, however, it's really useful to have access to a chart's interactive state in Python in order to use selections to drive other Python logic.

This is possible using [`JupyterChart`](https://altair-viz.github.io/user_guide/jupyter_chart.html), which is built on top of [Jupyter Widgets](https://ipywidgets.readthedocs.io/en/latest/) using [AnyWidget](https://anywidget.dev/).


Here is an example of accessing an interval selection from Python. Note that it's important to name the selection so that it's easy to look up in the `jchart.selections` object.

In [ ]:
brush = alt.selection_interval(name="brush")

chart = alt.Chart(cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.condition(brush, 'Cylinders:O', alt.value('grey')),
).add_params(brush)

jchart = alt.JupyterChart(chart)
jchart

In [ ]:
jchart.selections.brush.value

Here's a more complex example that uses the selection to filter a pandas DataFrame, then display that filtered pandas DataFrame below that chart in a separate widget.


In [ ]:
from ipywidgets import HTML, VBox

cars = data.cars()
brush = alt.selection_interval(name="brush")

chart_widget = alt.JupyterChart(alt.Chart(cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.condition(brush, 'Cylinders:O', alt.value('grey')),
).add_params(brush))

table_widget = HTML(value=cars.iloc[:0].to_html())

def on_select(change):
    sel = change.new.value
    if sel is None or 'Horsepower' not in sel:
        filtered = cars.iloc[:0]
    else:
        filter_query = (
            f"{sel['Horsepower'][0]} <= `Horsepower` <= {sel['Horsepower'][1]} and "
            f"{sel['Miles_per_Gallon'][0]} <= `Miles_per_Gallon` <= {sel['Miles_per_Gallon'][1]}"
        )
        filtered = cars.query(filter_query)

    table_widget.value = filtered.to_html()

chart_widget.selections.observe(on_select, ["brush"])

VBox([chart_widget, table_widget])

## Share interactive charts

A powerful feature of Vega-Altair is that once a chart is produced from Python, the interactive behavior of the chart does not depend on a Python kernel.

For example, you can design a Vega-Altair chart using Python, and then publish the same interactive chart on a website, even if that website does not have the ability to run Python code.  The reason this is possible is that Altair produces Vega-Lite/Vega specifications, and these rely on JavaScript, not Python.

The two most common ways to share interactive charts are the following.
* Share as html.
* Share within the Vega editor.

### Share as HTML

The following creates an html file which can be opened in a web browser to view the (fully interactive) chart:

In [ ]:
chart.save("output/fig.html")

#### Online vs. Offline

From the Vega-Altair documentation:

> By default, an HTML file generated by `chart.save('fig.html')` loads the necessary JavaScript dependencies from an online CDN location. This results in a small HTML file, but it means that an active internet connection is required in order to display the chart.
>
> As an alternative, the `inline=True` keyword argument may be provided to chart.save to generate an HTML file that includes all necessary JavaScript dependencies inline. This results in a larger file size, but HTML files generated this way do not require an active internet connection to display.

In [ ]:
chart.save("output/offline_fig.html", inline=True)

### Share in Vega editor
An entire Vega-Altair chart can be converted into a URL that opens the chart in the online Vega editor. This is what happens when you select "Open in Vega Editor" from the dropdown menu of a chart. This sharable URL can also be generated programmatically using the `chart.to_url()` method.

Clicking on the link output by the code below will open the chart in the Vega editor.

In [ ]:
chart.to_url()

> **Caution:** Sharing a URL link that opens the chart in the Vega editor can be a very convenient solution. However, it's important to be aware that the underlying data of the chart (e.g. the pandas DataFrame) is typically included within the produced Vega-Lite specification. This means that the URLs generated can be quite large, as they need to transmit not only the design options you chose for your chart but also all of the underlying data. Understanding this tradeoff can help you decide on the best approach for sharing your charts.

When working in a local development environment, you can also use the `chart.open_editor()` method to open the chart directly in the Vega editor. open_editor relies on running locally and having the ability to open a web browser. This isn't possible from inside a hosted notebook environment.

## Share static images

It is straightforward to share static images of your charts using the `save` method.  Supported formats include png, svg, and pdf.

### PNG

In [ ]:
chart.save("output/fig.png")

### SVG

In [ ]:
chart.save("output/fig.svg")

### PDF

In [ ]:
chart.save("output/fig.pdf")

## Embed in static documents

Vega-Altair charts can be embedded in larger static documents using a variety of technologies. Here are a few examples:

Quarto: https://quarto.org/docs/dashboards/data-display.html#altair

Jupyter Book: https://jupyterbook.org/en/stable/interactive/interactive.html#altair

LaTeX: Export to PDF and embed in LaTeX

# Dashboards

Vega-Altair is compatible with several popular Python dashboarding toolkits. Many of these toolkits support reading parameters from Vega-Altair charts (providing similar functionality to the `JupyterChart` class described above), allowing for interactive features like selecting data points to update other dashboard components.

See the [dashboards section](https://altair-viz.github.io/user_guide/display_frontends.html#dashboards) of the Vega-Altair documentation for more information.